In [3]:
# Jupyter 전용 -> 현재 커널(지금 콘다 env)에만 적용됩니다.
%pip install "numpy<2.0" --upgrade
# 권장 고정: %pip install numpy==1.26.4


Note: you may need to restart the kernel to use updated packages.


In [4]:
import sys, subprocess
# 버전 확인
subprocess.check_call([sys.executable, "-m", "pip", "show", "torch", "torchvision"])

# torchvision을 0.18.1(cu118)로 재설치
subprocess.check_call([sys.executable, "-m", "pip", "uninstall", "-y", "torchvision"])
subprocess.check_call([
    sys.executable, "-m", "pip", "install",
    "--index-url", "https://download.pytorch.org/whl/cu118",
    "torchvision==0.18.1"
])

# (선택) torchaudio도 맞추기 — torch 2.3.* ↔ torchaudio 2.3.*
# 이미 정상이라면 생략 가능
# subprocess.check_call([
#     sys.executable, "-m", "pip", "install",
#     "--index-url", "https://download.pytorch.org/whl/cu118",
#     "torchaudio==2.3.0"
# ])

print(">> 설치 완료. 커널을 재시작하고 학습 셀을 다시 실행하세요.")


KeyboardInterrupt: 

In [1]:
import torch, torchvision
print("torch      :", torch.__version__)      # 2.3.0 예상
print("torchvision:", torchvision.__version__) # 0.18.1 예상
import numpy, torch, torchvision
print("numpy:", numpy.__version__)        # 1.26.x 기대
print("torch :", torch.__version__)
print("tv    :", torchvision.__version__)


torch      : 2.3.1+cu118
torchvision: 0.18.1+cu118
numpy: 1.26.4
torch : 2.3.1+cu118
tv    : 0.18.1+cu118


In [2]:
import torch, platform
print("Python:", platform.python_version())
print("Torch :", torch.__version__, " CUDA build:", torch.version.cuda)
print("CUDA avail:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))


Python: 3.10.14
Torch : 2.3.1+cu118  CUDA build: 11.8
CUDA avail: True
GPU: NVIDIA GeForce RTX 4070 Laptop GPU


In [3]:
from pathlib import Path
DATA_DIR = Path(r"Red_reflector_detector.v4-annotatae_7_ver.yolov8")  # <-- 본인 경로
data_yaml = DATA_DIR / "data.yaml"
assert data_yaml.exists(), f"data.yaml 못 찾음: {data_yaml}"
print("Using:", data_yaml)


Using: Red_reflector_detector.v4-annotatae_7_ver.yolov8\data.yaml


In [4]:
from pathlib import Path
def count_pairs(split):
    img = list((DATA_DIR/split/"images").glob("*.*"))
    lbl = list((DATA_DIR/split/"labels").glob("*.txt"))
    return len(img), len(lbl)

for sp in ["train","valid","test"]:
    p = DATA_DIR/sp
    if p.exists():
        ni, nl = count_pairs(sp)
        print(f"{sp:5s}  images={ni:4d}  labels={nl:4d}")


train  images=1369  labels=1369
valid  images= 334  labels= 334


In [5]:
from pathlib import Path
import yaml

DATA_DIR = Path(r"Red_reflector_detector.v3-annotatae_3_ver.yolov8")  # 네 데이터 폴더
data_yaml = DATA_DIR / "data.yaml"

with open(data_yaml, "r", encoding="utf-8") as f:
    cfg = yaml.safe_load(f)

# 1) valid → val 매핑
if "val" not in cfg and "valid" in cfg:
    cfg["val"] = cfg["valid"]

# 2) 상대경로 정리: ../train/images  →  train/images  (같은 폴더 기준)
def fix(p):
    if isinstance(p, str) and p.startswith("../"):
        return p[3:]
    return p

for key in ["train", "val", "test"]:
    if key in cfg and cfg[key]:
        cfg[key] = fix(cfg[key])

# 3) test 폴더가 실제 없으면 yaml에서 제거(선택)
def exists_dir(rel_path):
    p = (DATA_DIR / rel_path).resolve()
    return p.exists()

if "test" in cfg and cfg["test"] and not exists_dir(cfg["test"]):
    print("[INFO] test 이미지 폴더가 없어 data.yaml에서 test 항목을 제거합니다.")
    cfg.pop("test", None)

# 4) 저장
with open(data_yaml, "w", encoding="utf-8") as f:
    yaml.safe_dump(cfg, f, sort_keys=False, allow_unicode=True)

print("✅ data.yaml 패치 완료:\n", yaml.safe_dump(cfg, sort_keys=False, allow_unicode=True))


✅ data.yaml 패치 완료:
 train: train/images
val: valid/images
nc: 1
names:
- Sticker
roboflow:
  workspace: fso-vk47d
  project: red_reflector_detector-tosx4
  version: 3
  license: CC BY 4.0
  url: https://universe.roboflow.com/fso-vk47d/red_reflector_detector-tosx4/dataset/3



In [6]:
from pathlib import Path

DATA_DIR = Path(r"Red_reflector_detector.v3-annotatae_3_ver.yolov8")
splits = []
for sp in ["train","val","valid","test"]:
    p = DATA_DIR/sp/"labels"
    if p.exists(): splits.append(p)

bad_cls, bad_box = [], []
for lbl_dir in splits:
    for txt in lbl_dir.glob("*.txt"):
        with open(txt, "r") as f:
            for ln, line in enumerate(f, 1):
                parts = line.strip().split()
                if len(parts) != 5: continue
                c = int(float(parts[0]))
                if c != 0:
                    bad_cls.append((txt, ln, c))
                try:
                    x, y, w, h = map(float, parts[1:])
                    if not (0 <= x <= 1 and 0 <= y <= 1 and 0 < w <= 1 and 0 < h <= 1):
                        bad_box.append((txt, ln, x, y, w, h))
                except:
                    bad_box.append((txt, ln, parts[1:]))

print(f"클래스 0 이외 라벨 수: {len(bad_cls)}")
print(f"[0~1] 범위 벗어난 bbox 수: {len(bad_box)}")
if bad_cls[:3]: print("예시(최대 3개):", bad_cls[:3])
if bad_box[:3]: print("예시(최대 3개):", bad_box[:3])


클래스 0 이외 라벨 수: 0
[0~1] 범위 벗어난 bbox 수: 0


In [7]:
from ultralytics import YOLO
model = YOLO("yolov8m.pt")
model.train(
    data=str(data_yaml),
    imgsz=640, epochs=80, batch=-1,
    device=0, amp=True, workers=4, cos_lr=True,
    patience=20, project="runs/detect", name="train_y8m_ud"
)


New https://pypi.org/project/ultralytics/8.3.202 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.201  Python-3.10.14 torch-2.3.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4070 Laptop GPU, 8188MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=Red_reflector_detector.v3-annotatae_3_ver.yolov8\data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=80, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001340507C220>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480